In [1]:
import os
import re
import json
from tqdm import tqdm

import pandas as pd

from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import get_openai_callback
from langchain_core.messages.system import SystemMessage
from langchain_core.messages.human import HumanMessage
from datasets import load_dataset

In [2]:
CAMBIO_DOLAR = 5.41
MODEL_NAME = "gpt-4o-mini"
INPUT_PRICE_PER_MILLION = 0.15
OUTPUT_PRICE_PER_MILLION = 0.6
FOLDER_PATH = "data/question-generator/base-train-dataset"

llm = ChatOpenAI(model=MODEL_NAME)

prompt_text = """
Dado o seguinte contexto:
```
{context}
```
Escreva um conjunto de perguntas e respostas sobre o contexto.
As perguntas devem ser tais que, concatenando as respostas, seja possível recuperar o conteúdo completo do contexto.
As perguntas deve ser autocontidas; ou seja, um leitor deve entendê-la sem neessariamente conhecer o contexto



# Formato de saída:
O resultado deve ser parseavel em json:

```
[
 {{"pergunta": "<<pergunta 1>>", "pergunta": "<<resposta 1>>"}},
 {{"pergunta": "<<pergunta 2>>", "pergunta": "<<resposta 2>>"}},
 ...
]
```
"""



In [3]:
def get_list(texto):
    resultado = re.search(r'\[([\s\S]*)\]', texto)
    if resultado:
        conteudo = resultado.group(1)
        return json.loads(f"[{conteudo}]")
    else:
        return "Nenhum conteúdo encontrado entre os colchetes."


In [4]:
def get_questions(context):
    chat_prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="You are a helpful assistant."),
        HumanMessage(content=prompt_text.format(context=context))
    ])
    
    chain = chat_prompt | llm
    with get_openai_callback() as cb:
        response = chain.invoke(dict(context=context))
        preco_em_real = CAMBIO_DOLAR * (INPUT_PRICE_PER_MILLION * cb.prompt_tokens+ OUTPUT_PRICE_PER_MILLION*cb.completion_tokens) / 1_000_000
        # print(f"preco = {preco_em_real}")

    
    try:
        _list = get_list(response.content)
        error = False
    except:
        _list = None
        error = True
        
    return response.content, _list, preco_em_real, error


In [5]:
def run_llm_data_generator(row):
    response, formatted_response, preco_real, error = get_questions(row.context)

    df = pd.DataFrame(
        [
            {
                "id": row.id,
                "context": row.context,
                "questions": response,
                "formatted_questions": formatted_response,
                "price": preco_real,
            }
        ]
    )

    df.to_parquet(f"{FOLDER_PATH}/{row.id}.parquet")
    return response, preco_real, error

# Loop

In [6]:
dataset = load_dataset("nunorc/squad_v1_pt")
dataset = dataset["train"].shuffle()


In [7]:
df = dataset.to_pandas()
df["context_len"] = df["context"].str.len()
df = df[df["context_len"] < 1000]

df

,id,title,context,question,answers,context_len
1,56d2a28059d6e41400146172,Buddhism,O budismo oferece muitas oportunidades para es...,O conceito budista de originação dependente fo...,"{'text': ['científico'], 'answer_start': [629]}",681
3,5728464e3acd2414000df854,War_on_Terror,O conflito no norte do Mali começou em janeiro...,Em que grupo estavam os radicais do Mali afili...,"{'text': ['al-Qaeda'], 'answer_start': [92]}",551
5,57284b0c4b864d19001648dd,LaserDisc,O suporte do LaserDisc a múltiplas faixas de á...,"Qual recurso, exclusivo do LaserDisc, possibil...",{'text': ['suporte para várias faixas de áudio...,578
6,5726adaff1498d1400e8e6ee,Germans,As pessoas que falam alemão como primeira líng...,Residência que fala alemão como língua materna...,"{'text': ['mais alemão'], 'answer_start': [0]}",679
8,5727849df1498d1400e8fa86,FA_Cup,"Em 2003, a FA tomou a decisão de usar permanen...",Onde está a taça da FA hoje?,"{'text': ['Em 2003, a FA tomou a decisão de us...",669
...,...,...,...,...,...,...
87594,56e02b8d231d4119001abf76,Comics,Revistas ilustradas para expatriados ocidentai...,Quem introduziu tiras satíricas para o Japão?,"{'text': ['Expatriados ocidentais'], 'answer_s...",744
87595,5730ec8305b4da19006bcc49,United_States_Air_Force,"Devido ao sequestro do orçamento em 2013, a US...",Onde muitos desses pilotos da Força Aérea dos ...,{'text': ['carreiras nas linhas aéreas comerci...,636
87596,5729141f6aef051400154a3b,Software_testing,Os desenvolvedores de software não podem testa...,Em que consiste o uso de testes combinatórios?,{'text': ['obter maior cobertura de teste com ...,605
87597,5726bb5b708984140094cf84,Pesticide,"Nos Estados Unidos, a Agência de Proteção Ambi...",Quem é protegido pelos regulamentos da EPA?,"{'text': ['humanos ou o meio ambiente'], 'answ...",979


In [27]:
%%time

SAMPLE_SIZE = 100
MAX_TOTAL_PRICE = 7.
MAX_ERROR_COUNT = int(0.1*SAMPLE_SIZE)


sample = df.sample(SAMPLE_SIZE)

errors_count = 0

total_price = pd.read_parquet(FOLDER_PATH)['price'].sum()


for index, row in tqdm(sample.iterrows()):
    if (errors_count > MAX_ERROR_COUNT) or (total_price > MAX_TOTAL_PRICE):
        print(f"Forcing break")
        print(f"errors_count = {errors_count} | MAX_ERROR_COUNT = {MAX_ERROR_COUNT} | errors_count >= MAX_ERROR_COUNT =", errors_count >= MAX_ERROR_COUNT)
        print(f"total_price = {total_price} | MAX_TOTAL_PRICE = {MAX_TOTAL_PRICE} | total_price >= MAX_TOTAL_PRICE =", total_price >= MAX_TOTAL_PRICE)
        break
    
    try:
        response, preco_real, error = run_llm_data_generator(row)
        total_price += preco_real
        if error:
            errors_count += 1
    except Exception as err:
        err_message = f"[ERRO] {err}"
        print(err_message)
        if "Incorrect API key provided" in err_message:
            print(f"Forcing break")
            break
        errors_count += 1

    if (index+1) % 10 == 0:
        try:
            temp =  pd.read_parquet(FOLDER_PATH)
            print(f"Preço atá agora: {temp['price'].sum()}")
        except:
            print("Não foi possível ler o parquet. Corrija isso antes de prosseguir")
            break


3it [00:28,  8.91s/it]

Preço atá agora: 0.277661217


5it [00:41,  7.23s/it]

Preço atá agora: 0.2810557215


14it [01:29,  4.66s/it]

...

Preço atá agora: 0.335178714


51it [04:35,  6.07s/it]

Preço atá agora: 0.338275398


100it [08:36,  5.16s/it]

Preço atá agora: 0.402421227
CPU times: user 2.27 s, sys: 256 ms, total: 2.53 s
Wall time: 8min 36s


In [28]:
temp =  pd.read_parquet(FOLDER_PATH)
print(f"Preço atá agora: {temp['price'].sum()}")

Preço atá agora: 0.402421227


In [32]:
temp.sample()["formatted_questions"].iloc[0]

array([{'pergunta': 'Quais são os esportes mais populares em Melbourne?', 'resposta': 'As regras australianas de futebol e críquete são os esportes mais populares em Melbourne.'},
       {'pergunta': 'Qual é o lar espiritual dos esportes na Austrália?', 'resposta': 'Melbourne é considerado o lar espiritual dos dois esportes na Austrália.'},
       {'pergunta': 'Quando foi disputado o primeiro jogo oficial de críquete do teste e onde?', 'resposta': 'O primeiro jogo oficial de críquete do teste foi disputado no Melbourne Cricket Ground em março de 1877.'},
       {'pergunta': 'Quando as origens do futebol australiano podem ser rastreadas?', 'resposta': 'As origens do futebol australiano podem ser encontradas nos jogos disputados ao lado do MCG em 1858.'},
       {'pergunta': 'Onde está sediada a Australian Football League?', 'resposta': 'A Australian Football League está sediada no Docklands Stadium.'},
       {'pergunta': 'Quantas equipes da AFL estão localizadas na área metropolitana d

100 -> 7min, R$ 0,13

In [19]:
temp['price'].mean() * 100

0.12830162785714286

In [22]:
50 * 7 / 60

5.833333333333333

In [24]:
50 * 0.13

6.5